Notebook to train a custom FinBERT model on the binary AINI dataset to annotate WSJ articles.

In [1]:
from pathlib import Path
import random
import sqlite3
import pandas as pd
import numpy as np
import torch
import os
from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, set_seed
from transformers import DataCollatorWithPadding, EarlyStoppingCallback,AutoConfig, get_scheduler,BertModel
from IPython.display import display, Markdown 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn
from transformers.modeling_outputs import SequenceClassifierOutput
import sys
from pathlib import Path
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
import spacy
import nltk

try:
    nlp = spacy.load("en_core_web_sm")
except:
    nltk.download("punkt")
    import en_core_web_sm
    nlp = en_core_web_sm.load()

# append model path
notebook_dir = Path.cwd()
project_root = notebook_dir.parent  
sys.path.insert(0, str(project_root))  

from src.modelling.CustomFinBERT import CustomFinBERT
from src.modelling.ai_windows import extract_multiple_ai_snippets_with_context


ModuleNotFoundError: No module named 'datasets'

In [2]:
# Set project root, assumes being in /notebooks
project_root = Path.cwd().parent

# Path to annotated cvs
articles_dir = project_root / "data" / "processed" / "articles" / "annotated_subsample_WSJ_final.csv"

# read and merge annotated cvs
df = pd.read_csv(articles_dir)

# verify merge
print(f" {len(df)} total rows.")
display(df.head(5))
print(set(df.hype_level.values))
print(df.columns)

 1018 total rows.


,article_id,title,sub_title,cleaned_corpus,label_ai_related,hype_level,modified,hype_level_change,index_id,scanned_time,corpus,section,date
0,28537,Apple iPhone Sales Slump in China Amid Huawei’...,Huawei’s sales were helped by the successful l...,"April 23, 651 am. ET 2 min smartphone sales in...",1,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,31336,AI Is Helping Scammers Outsmart You—and Your Bank,Your ‘spidey sense’ is no match for the new wa...,"ILLUSTRATION. MARK. HARRIS,. ISTOCK 7. June 22...",1,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,29569,Alibaba Cloud Trims Prices for International C...,Alibaba Cloud said the cuts were aligned with ...,Alibaba. Cloud said the cuts were aligned with...,1,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,18063,Nvidia to Replace Intel in Dow Jones Industria...,The swap reflects their reversal of fortunes w...,The latest news analysis. MARKETS. FINANCE. Nv...,1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,29872,The Man Whose Musings Fuel Elon Musk’s Nightmares,A marketing professor in Canada has formed a b...,The. Man. Whose. Musings. Fuel. Elon. Musk's. ...,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


{np.float64(0.0), np.float64(1.0), np.float64(2.0), np.float64(3.0)}
Index(['article_id', 'title', 'sub_title', 'cleaned_corpus',
       'label_ai_related', 'hype_level', 'modified', 'hype_level_change',
       'index_id', 'scanned_time', 'corpus', 'section', 'date'],
      dtype='object')


In [3]:
# ensure integrity
print(df.corpus.isna().sum())
labeled_df = df.copy()
labeled_df.loc[labeled_df["corpus"].isna(), "corpus"] = df.loc[df["corpus"].isna(), "cleaned_corpus"]

# verify fix
print(labeled_df.corpus.isna().sum())

218
0


In [4]:
# rename column
labeled_df.rename(columns={"hype_level": "label"}, inplace=True)

# binary label creation
labeled_df["label"] = labeled_df["label"].apply(lambda x: 1 if x in [1.0, 2.0, 3.0] else 0)

# merging cleaned corpus and title on corpus
labeled_df["corpus"] = "Title: " + labeled_df["title"] + "\n\n" + labeled_df["corpus"]

#  Create class weights
labels = labeled_df["label"]
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)


# Convert to HF Dataset
hf_labeled_df = Dataset.from_pandas(labeled_df)

In [5]:
def custom_truncate(batch, tokenizer):
    encoded = tokenizer(batch["corpus"], add_special_tokens=False)
    input_ids = encoded["input_ids"] # word tokens mapped to ids
    attention_mask = encoded["attention_mask"] # 0 if padded

    truncated_input_ids = []
    truncated_attention_mask = []

    for ids, mask in zip(input_ids, attention_mask):
        
        # limit input length
        if len(ids) > 510:
            new_ids = ids[:128] + ids[-382:]
            new_mask = mask[:128] + mask[-382:]
        else:
            new_ids = ids
            new_mask = mask

        # Add [CLS] and [SEP]
        new_ids = [tokenizer.cls_token_id] + new_ids + [tokenizer.sep_token_id]
        new_mask = [1] + new_mask + [1]

        pad_len = 512 - len(new_ids)
        new_ids += [tokenizer.pad_token_id] * pad_len
        new_mask += [0] * pad_len

        truncated_input_ids.append(new_ids)
        truncated_attention_mask.append(new_mask)

    return {"input_ids": truncated_input_ids, "attention_mask": truncated_attention_mask}


In [6]:
# Set seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
set_seed(seed)

# Convert to pandas and extract AI-focused snippets
df_labeled = hf_labeled_df.to_pandas()
df_labeled = extract_multiple_ai_snippets_with_context(df_labeled, text_col='corpus', output_col='ai_window',context_window=2)

# Stratified split: 70% train, 30% temp
train_df, temp_df = train_test_split(
    df_labeled,
    test_size=0.3,
    stratify=labeled_df["label"],
    random_state=seed
)

# Stratified split: 2/3 test, 1/3 eval from temp (→ 20% / 10%)
eval_df, test_df  = train_test_split(
    temp_df,
    test_size=1/3,
    stratify=temp_df["label"],
    random_state=seed
)


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

# Tokenization function
def tokenize(example):
    return tokenizer(
        example["ai_window"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))
eval_dataset = Dataset.from_pandas(eval_df.reset_index(drop=True))

# fixing label for tensors
def fix_labels(example):
    return {"labels": int(example["label"])}

train_dataset = train_dataset.map(tokenize)
train_dataset = train_dataset.map(fix_labels)

eval_dataset = eval_dataset.map(tokenize)
eval_dataset = eval_dataset.map(fix_labels)

test_dataset = test_dataset.map(tokenize)
test_dataset = test_dataset.map(fix_labels)



# prepare dictionary for trainer
dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
    "eval": eval_dataset
})

# create class weights
labels = labeled_df["label"]
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

Map: 100%|██████████| 102/102 [00:00<00:00, 3200.23 examples/s]


In [7]:
# Define model path and label mappings
model_path = "ProsusAI/finbert"
id2label = {0: "No narrative", 1: "narrative"}
label2id = {v: k for k, v in id2label.items()}

# Load and customize config
config = AutoConfig.from_pretrained(
    model_path,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1
)

# Load pretrained backbone
backbone = BertModel.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights_tensor = class_weights_tensor.to(device)

# Instantiate the model
model = CustomFinBERT(backbone, class_weights_tensor, config).to(device)

print(f"Model is on device: {next(model.parameters()).device}")

# Format datasets
columns = ["input_ids", "attention_mask", "labels"]
train_dataset.set_format(type="torch", columns=columns)
test_dataset.set_format(type="torch", columns=columns)
eval_dataset.set_format(type="torch", columns=columns)

Model is on device: cuda:0


In [8]:
# narrativerparameters 
lr = 2e-5
batch_size = 4
num_epochs = 15

# data collator for padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir= project_root / "models" / "FINBERT_Binary",
    learning_rate=lr,
    logging_dir= project_root / "models" / "FINBERT_Binary",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    load_best_model_at_end=True,
    save_total_limit=None
)

In [9]:
# Freeze all parameters in the FinBERT encoder
for name, param in model.backbone.named_parameters():
    param.requires_grad = False

# Unfreeze the classification head and optional with ( name or "encoder.layer.11" in ) the last encoder layer (layer.11)
for name, param in model.named_parameters():
    if "classifier" or "encoder.layer.11" in name:
        param.requires_grad = True


# Print a summary of how many parameters are trainable
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params:,} / {total_params:,} ({trainable_params / total_params:.2%})")

# Optionally, list all parameter names that are currently trainable
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"[✓] {name}")


Trainable parameters: 109,483,778 / 109,483,778 (100.00%)
[✓] backbone.embeddings.word_embeddings.weight
[✓] backbone.embeddings.position_embeddings.weight
[✓] backbone.embeddings.token_type_embeddings.weight
[✓] backbone.embeddings.LayerNorm.weight
[✓] backbone.embeddings.LayerNorm.bias
[✓] backbone.encoder.layer.0.attention.self.query.weight
[✓] backbone.encoder.layer.0.attention.self.query.bias
[✓] backbone.encoder.layer.0.attention.self.key.weight
[✓] backbone.encoder.layer.0.attention.self.key.bias
[✓] backbone.encoder.layer.0.attention.self.value.weight
[✓] backbone.encoder.layer.0.attention.self.value.bias
[✓] backbone.encoder.layer.0.attention.output.dense.weight
[✓] backbone.encoder.layer.0.attention.output.dense.bias
[✓] backbone.encoder.layer.0.attention.output.LayerNorm.weight
[✓] backbone.encoder.layer.0.attention.output.LayerNorm.bias
[✓] backbone.encoder.layer.0.intermediate.dense.weight
[✓] backbone.encoder.layer.0.intermediate.dense.bias
[✓] backbone.encoder.layer.0.ou

In [10]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro"),
        "precision_macro": precision_score(labels, preds, average="macro", zero_division=0),
        "recall_macro": recall_score(labels, preds, average="macro", zero_division=0),
        "f1_weighted": f1_score(labels, preds, average="weighted"),
        "precision_weighted": precision_score(labels, preds, average="weighted", zero_division=0),
        "recall_weighted": recall_score(labels, preds, average="weighted", zero_division=0),
    }


In [11]:
# Sanity check on label values
unique_labels = set(train_dataset["labels"].tolist())
print("Unique labels in training data:", unique_labels)
assert all(label in [0, 1] for label in unique_labels), "Found unexpected label values!"

# Ensure GPU is used   
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

Unique labels in training data: {0, 1}


In [12]:
# train them model
    
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset, # 70% training set
    eval_dataset=eval_dataset, # 20% training set
    tokenizer=tokenizer,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True),
    compute_metrics=compute_metrics, # metrics as defined above
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)],
)

trainer.train()

C:\Users\PC\AppData\Local\Temp\ipykernel_12148\1632590623.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision Macro,Recall Macro,F1 Weighted,Precision Weighted,Recall Weighted
1,0.343200,0.477074,0.901961,0.899160,0.895261,0.906052,0.902620,0.906084,0.901961
2,0.331900,0.524069,0.901961,0.899160,0.895261,0.906052,0.902620,0.906084,0.901961
3,0.264800,0.550139,0.897059,0.893937,0.890282,0.899880,0.897683,0.900447,0.897059
4,0.194400,0.574569,0.897059,0.893937,0.890282,0.899880,0.897683,0.900447,0.897059
5,0.155800,0.651273,0.892157,0.889076,0.885296,0.895815,0.892882,0.896379,0.892157


TrainOutput(global_step=890, training_loss=0.2580179171615772, metrics={'train_runtime': 572.6048, 'train_samples_per_second': 18.652, 'train_steps_per_second': 4.663, 'total_flos': 0.0, 'train_loss': 0.2580179171615772, 'epoch': 5.0})

In [13]:
# Define save path
project_root = Path.cwd().parent
save_dir = project_root / "models" / "FinBERT_Binary" 
save_dir.mkdir(parents=True, exist_ok=True)

# Save model, tokenizer and configd
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)
config.save_pretrained(save_dir)  

# Extract log history from training
log_history = trainer.state.log_history

# Convert to DataFrame
log_df = pd.DataFrame(log_history)

# Filter out entries without epoch information
log_df = log_df[log_df['epoch'].notna()].copy()

# Save full training logs
log_df.to_csv(f"{save_dir}/full_training_metrics.csv", index=False)
#print(log_df)

# Save optimizer and scheduler state
torch.save(trainer.optimizer.state_dict(), f"{save_dir}/optimizer.pt")
torch.save(trainer.lr_scheduler.state_dict(), f"{save_dir}/scheduler.pt")
torch.save(class_weights_tensor, save_dir / "class_weights.pt")

# Save config details
with open(f"{save_dir}/config_summary.txt", "w") as f:
    f.write(f"Model: {save_dir}\n")
    f.write(f"Learning Rate: {training_args.learning_rate}\n")
    f.write(f"Epochs: {training_args.num_train_epochs}\n")
    f.write(f"Batch Size: {training_args.per_device_train_batch_size}\n")
    f.write("Tokenizer: ProsusAI/finbert\n")
    f.write("Special preprocessing: large_ai_snippets_with_context=2\n")
    f.write("Additional finetuning: encoder.layer.11 \n")
    f.write("Label mapping: {0: 'No narrative', 1: 'narrative'}\n")

# Run prediction on eval set (not test set)
eval_output = trainer.predict(eval_dataset)

# Extract predictions and true labels
eval_preds = np.argmax(eval_output.predictions, axis=1)
eval_labels = eval_output.label_ids

# Create DataFrame with predictions
eval_results_df = pd.DataFrame({
    "text": eval_dataset["ai_window"],
    "true_label": eval_labels,
    "predicted_label": eval_preds
})

# Map label IDs to names
id2label =  {0: 'No narrative', 1: 'narrative'}
eval_results_df["true_label_name"] = eval_results_df["true_label"].map(id2label)
eval_results_df["predicted_label_name"] = eval_results_df["predicted_label"].map(id2label)

# Save predictions
eval_results_df.to_csv(f"{save_dir}/eval_predictions.csv", index=False)

# Save evaluation classification report
report = classification_report(eval_labels, eval_preds, target_names=list(id2label.values()))
with open(f"{save_dir}/eval_classification_report.txt", "w") as f:
    f.write(report)

print(report)

              precision    recall  f1-score   support

No narrative       0.95      0.89      0.92       123
   narrative       0.84      0.93      0.88        81

    accuracy                           0.90       204
   macro avg       0.90      0.91      0.90       204
weighted avg       0.91      0.90      0.90       204



In [14]:
#Run prediction
output = trainer.predict(eval_dataset)

# Extract predicted and true labels
preds = np.argmax(output.predictions, axis=1)
true = output.label_ids

# Define label mappings
id2label = {0: "No narrative", 1: "narrative"}

# Create full DataFrame
results_df = pd.DataFrame({
    "text": eval_dataset["ai_window"],
    "true_label": true,
    "predicted_label": preds,
    "true_label_name": [id2label[i] for i in true],
    "predicted_label_name": [id2label[i] for i in preds]
})

# Filter misclassified samples
misclassified_df = results_df[results_df["true_label"] != results_df["predicted_label"]]

# Save to CSV
misclassified_df.to_csv("misclassified_eval_samples.csv", index=False)

# Adjust pandas display options for full text visibility
pd.set_option("display.max_colwidth", None)
# Save model state dict manually
torch.save(model.state_dict(), save_dir / "pytorch_model.bin")

# Save config and tokenizer
config.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('c:\\Users\\PC\\Desktop\\Masterarbeit\\AI_narrative_index\\models\\FinBERT_Binary\\tokenizer_config.json',
 'c:\\Users\\PC\\Desktop\\Masterarbeit\\AI_narrative_index\\models\\FinBERT_Binary\\special_tokens_map.json',
 'c:\\Users\\PC\\Desktop\\Masterarbeit\\AI_narrative_index\\models\\FinBERT_Binary\\vocab.txt',
 'c:\\Users\\PC\\Desktop\\Masterarbeit\\AI_narrative_index\\models\\FinBERT_Binary\\added_tokens.json',
 'c:\\Users\\PC\\Desktop\\Masterarbeit\\AI_narrative_index\\models\\FinBERT_Binary\\tokenizer.json')

Just for FINAL evaluation:

In [15]:
# Extract log history
log_history = trainer.state.log_history

# Convert to DataFrame
log_df = pd.DataFrame(log_history)

# 3. Filter out irrelevant entries (like those without epoch info)
log_df = log_df[log_df['epoch'].notna()].copy()

# Save model and tokenizer
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

# Save full logs with all metrics
log_df.to_csv(f"{save_dir}/full_training_metrics.csv", index=False)

# Print or display metrics
print(log_df)

# Save optimizer and learning rate scheduler state
torch.save(trainer.optimizer.state_dict(), f"{save_dir}/optimizer.pt")
torch.save(trainer.lr_scheduler.state_dict(), f"{save_dir}/scheduler.pt")

# write model details to txt
with open(f"{save_dir}/config_summary.txt", "w") as f:
    f.write(f"Model: {save_dir}\n")
    f.write(f"Learning Rate: {training_args.learning_rate}\n")
    f.write(f"Epochs: {training_args.num_train_epochs}\n")
    f.write(f"Batch Size: {training_args.per_device_train_batch_size}\n")
    f.write("Tokenizer: ProsusAI/finbert\n")
    f.write("Special preprocessing: large_ai_snippets_with_context=2\n")
    f.write("Additional finetuning:  ")
    f.write("Label mapping: {0: 'No narrative', 1: 'narrative'}\n")

# Run prediction on test set
test_output = trainer.predict(test_dataset)

# Extract predictions and true labels
test_preds = np.argmax(test_output.predictions, axis=1)
test_labels = test_output.label_ids

# Save raw predictions and true labels to CSV
test_results_df = pd.DataFrame({
    "text": test_dataset["ai_window"],
    "true_label": test_labels,
    "predicted_label": test_preds
})

# map to label names
id2label = {0: "No narrative", 1: "narrative"}
test_results_df["true_label_name"] = test_results_df["true_label"].map(id2label)
test_results_df["predicted_label_name"] = test_results_df["predicted_label"].map(id2label)

# Save to CSV
test_results_df.to_csv(f"{save_dir}/test_predictions.csv", index=False)

# Save classification report (precision, recall, f1)
report = classification_report(test_labels, test_preds, target_names=list(id2label.values()))
with open(f"{save_dir}/test_classification_report.txt", "w") as f:
    f.write(report)    

      loss  grad_norm  learning_rate  epoch  step  eval_loss  eval_accuracy  \
0   0.3432   0.189552       0.000019    1.0   178        NaN            NaN   
1      NaN        NaN            NaN    1.0   178   0.477074       0.901961   
2   0.3319   0.187772       0.000017    2.0   356        NaN            NaN   
3      NaN        NaN            NaN    2.0   356   0.524069       0.901961   
4   0.2648   0.092782       0.000016    3.0   534        NaN            NaN   
5      NaN        NaN            NaN    3.0   534   0.550139       0.897059   
6   0.1944   0.103910       0.000015    4.0   712        NaN            NaN   
7      NaN        NaN            NaN    4.0   712   0.574569       0.897059   
8   0.1558   0.034760       0.000013    5.0   890        NaN            NaN   
9      NaN        NaN            NaN    5.0   890   0.651273       0.892157   
10     NaN        NaN            NaN    5.0   890        NaN            NaN   

    eval_f1_macro  eval_precision_macro  eval_recal

In [16]:
print(report)

              precision    recall  f1-score   support

No narrative       0.98      0.89      0.93        61
   narrative       0.85      0.98      0.91        41

    accuracy                           0.92       102
   macro avg       0.92      0.93      0.92       102
weighted avg       0.93      0.92      0.92       102

